In [4]:
# 使用的框架paddle
# 相关框架的加载
import paddle
from paddle.nn import Linear
import paddle.nn.functional as F
import os
import gzip
import json
import random
import numpy as np
from visualdl import LogWriter

In [5]:
import paddle
from paddle.vision.transforms import Normalize

def get_MNIST_dataloader():
    # 定义图像归一化处理方法，这里的CHW指图像格式需为 [C通道数，H图像高度，W图像宽度]
    transform = Normalize(mean=[127.5], std=[127.5], data_format='CHW')
    # 下载数据集并初始化 DataSet
    train_dataset = paddle.vision.datasets.MNIST(mode='train', transform=transform)
    test_dataset = paddle.vision.datasets.MNIST(mode='test', transform=transform)

    # 定义并初始化数据读取器
    train_loader = paddle.io.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=1, drop_last=True)
    test_loader = paddle.io.DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=1, drop_last=False)

    return train_loader, test_loader

In [6]:
train_loader, test_loader = get_MNIST_dataloader()

e:\SoftWare\Program\Anaconda\envs\d2l\lib\site-packages\paddle\io\reader.py:433: UserWarning: DataLoader with multi-process mode is not supported on MacOs and Windows currently. Please use signle-process mode with num_workers = 0 instead
  warnings.warn(


In [7]:
class SOFTMAX_NET(paddle.nn.Layer):
    def __init__(self):
        super(SOFTMAX_NET, self).__init__()
        self.fc1 = Linear(in_features=784, out_features=10)
        
    def forward(self, inputs):
        inputs = paddle.reshape(inputs, [inputs.shape[0], 784])
        outputs1 = self.fc1(inputs)
        outputs_final = F.softmax(outputs1)
        return outputs_final


In [8]:
# 数据载入
logwriter = LogWriter(logdir='./log')
class MNISTDataset():
  def __init__(self, mode='train'):
    self.mnist_data = paddle.vision.datasets.MNIST(mode=mode)

  def __getitem__(self, idx):
    data, label = self.mnist_data[idx]
    data = np.reshape(data, [1, 28, 28]).astype('float32') / 255
    label = np.reshape(label, [1]).astype('int64')
    return (data, label)

  def __len__(self):
    return len(self.mnist_data)

# 查看 9 张输入的训练图像的样例
dataset = MNISTDataset(mode='train')
image_matrix = []
for i in range(9):
  image, label = dataset[i]
  # 将 dataset 中的 CHW 排列的图像转换成 HWC 排列再写入 VisualDL
  image_matrix.append(image.transpose([1,2,0]))
# 将九张输入图像合成长宽相同的图像网格，即 3X3 的图像网格
logwriter.add_image_matrix(tag='input_images', step=1, imgs=image_matrix, rows=-1)


In [13]:
def train(model):
    model.train()
    train_batchs_per_epoch = len(train_loader)
    # 训练参数
    opt = paddle.optimizer.SGD(learning_rate=0.05, parameters=model.parameters())
    EPOCH_NUM = 10
    losses = []
    acc_test = []
    losses_test = []
    # 训练
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            # 数据准备
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.to_tensor(labels)

            # 前向训练
            predicts = model(images)

            # 交叉熵损失
            loss = F.cross_entropy(predicts, labels)
            avg_loss = paddle.mean(loss)

            #记录当前训练 Loss 到 VisualDL
            logwriter.add_scalar("train_avg_loss", value=avg_loss.numpy(), step=batch_id+epoch_id*(train_batchs_per_epoch))

            #每训练200批次的数据，打印下当前Loss的情况
            if batch_id % 200 == 0:
                losses.append(avg_loss.numpy()[0])
                print("epoch: {}, batch: {}, loss is: {}".format(epoch_id, batch_id, avg_loss.numpy()))
            
            # 反向
            avg_loss.backward()
            opt.step()
            opt.clear_grad()
        
        for batch_id, data in enumerate(test_loader()):
            model.eval()
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.to_tensor(labels)

            predicts = model(images)

            loss = F.cross_entropy(predicts, labels)
            acc = paddle.metric.accuracy(predicts, labels)

            acc_test.append(acc.numpy())
            losses_test.append(loss)
        test_avg_loss = np.mean(losses_test)
        test_avg_acc = np.mean(acc_test)
        print("[validation]After epoch {}: accuracy/loss: {}/{}".format(epoch_id,test_avg_acc , test_avg_acc))
        logwriter.add_scalar("test_avg_loss", value=test_avg_loss, step=epoch_id)
        logwriter.add_scalar("test_avg_acc", value=test_avg_acc, step=epoch_id)


    paddle.save(model.state_dict(), 'mnist_softmax.pdparams')

    return losses

In [14]:
model_softmax = SOFTMAX_NET()
# 网络结构
paddle.jit.save(model_softmax, './mnist_softmax.pdparams', [paddle.static.InputSpec([-1,1,28,28])])
losses_softmax = train(model_softmax)

e:\SoftWare\Program\Anaconda\envs\d2l\lib\site-packages\paddle\fluid\variable_index.py:591: UserWarning: Warning: In Tensor '__getitem__', if the number of scalar elements in the index is equal to the rank of the Tensor, the output should be 0-D. In order to be consistent with the behavior of previous versions, it will be processed to 1-D. But it is not correct and will be removed in release 2.6. If 1-D is still wanted, please modify the index element from scalar to slice (e.g. 'x[i]' => 'x[i:i+1]').
  warnings.warn(


epoch: 0, batch: 0, loss is: [2.3194082]
epoch: 0, batch: 200, loss is: [1.8708612]
epoch: 0, batch: 400, loss is: [1.882052]
epoch: 0, batch: 600, loss is: [1.8606933]
epoch: 0, batch: 800, loss is: [1.7190285]
[validation]After epoch 0: accuracy/loss: 0.6634156107902527/0.6634156107902527
epoch: 1, batch: 0, loss is: [1.7031863]
epoch: 1, batch: 200, loss is: [1.7168865]
epoch: 1, batch: 400, loss is: [1.8453493]
epoch: 1, batch: 600, loss is: [1.7943172]
epoch: 1, batch: 800, loss is: [1.7291168]
[validation]After epoch 1: accuracy/loss: 0.7030752301216125/0.7030752301216125
epoch: 2, batch: 0, loss is: [1.7218847]
epoch: 2, batch: 200, loss is: [1.6757426]
epoch: 2, batch: 400, loss is: [1.6252906]
epoch: 2, batch: 600, loss is: [1.6606683]
epoch: 2, batch: 800, loss is: [1.6534338]
[validation]After epoch 2: accuracy/loss: 0.7413747310638428/0.7413747310638428
epoch: 3, batch: 0, loss is: [1.7231197]
epoch: 3, batch: 200, loss is: [1.6592734]
epoch: 3, batch: 400, loss is: [1.7204

In [11]:
def evaluation(model, datasets):
    model.eval()

    acc_set = list()
    for batch_id, data in enumerate(datasets()):
        images, labels = data
        images = paddle.to_tensor(images)
        labels = paddle.to_tensor(labels)
        pred = model(images)   # 获取预测值
        acc = paddle.metric.accuracy(input=pred, label=labels)
        acc_set.extend(acc.numpy())
    
    # #计算多个batch的准确率
    acc_val_mean = np.array(acc_set).mean()
    return acc_val_mean

In [12]:
acc_of_softmax = evaluation(model_softmax, test_loader)
print(f'acc_of_softmax:{acc_of_softmax}')

acc_of_softmax:0.7382563948631287
